# DDI-ML. Relation Extraction

Setup
cd into ../stanford-corenlp-4.2.0 and run

`java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer`

In [1]:
from os import listdir
from xml.dom.minidom import parse
import networkx
# import nltk CoreNLP module (just once)
from nltk.parse.corenlp import CoreNLPDependencyParser
# connect to your CoreNLP server (just once)
corenlp_parser = CoreNLPDependencyParser(url="http://localhost:9000")

from evaluator import *

## Feature Extractor

In [2]:
def get_offsets(word, s):
    '''
    Task:
        Given a word and sentence, returns its starting end ending index in the sentence.
    
    Input:
        word: word to find offsets for
        s: sentence containing the word
    
    Output:
        Returns a tuple containing the start and end offset.
    '''
    start = s.find(word)
    end = start + len(word) - 1
    return start, end

In [3]:
def preprocess(s):
    '''
    Task:
        Helper function
    '''
    # because otherwise CoreNLP throws 500
    return s.replace("%", "<percentage>")

In [4]:
def analyze(s):
    '''
    Task:
        Given one sentence, sends it to CoreNLP to obtain the tokens, tags,
        and dependency tree. It also adds the start/end offsets to each token.
    
    Input:
        s: string containing the text for one sentence
    
    Output:
        Returns the nltk DependencyGraph object produced by CoreNLP, enriched with token  offsets.

    '''
    s = s.replace("%", "<percentage>")
    tree, = corenlp_parser.raw_parse(s)
    for n in tree.nodes.items():
        node = n[1]
        if node['word']:
            start, end = get_offsets(node['word'], s)
            node['start'] = start
            node['end'] = end
            
    return tree

In [5]:
def find_entity_in_tree(eid, entities, tree):
    start_e1 = entities[eid][0]
    end_e1 = entities[eid][1]
    
    for n in tree.nodes.items():
        node = n[1]
        if node['word'] and node['start'] == int(start_e1): # and node['end'] == int(end_e1)):
            return node
    
    # TODO: handle two-word entities like "beta-endorphin"
    # We can extract here word, lemma, tag

In [6]:
def find_head(tree, entity):
    for n in tree.nodes.items():
            node = n[1]
            if  node['address'] == entity['head']:
                return node
    # We can extract here word, lemma, tag

In [7]:
# file for initial checks
file = '/Users/mponsclo/Documents/Master/labAHLT/data/train/3155550.xml'
file = '../../labAHLT/data/train/Dexamethasone_ddi.xml'
tree = parse(file)
sentences = tree.getElementsByTagName("sentence")
for s in sentences:
    sid = s.attributes["id"].value
    stext = s.attributes["text"].value
    
    entities = {}
    ents = s.getElementsByTagName("entity")
    for e in ents:
        eid = e . attributes["id"].value
        entities[eid] = e.attributes["charOffset"].value.split("-")
    if len(entities) > 1: analysis = analyze(stext)
    
    pairs = s.getElementsByTagName("pair")
    for p in pairs:
        # get ground truth
        ddi = p.attributes["ddi"].value
        dditype = p.attributes["type"].value if ddi == "true" else "null"
            
        # target entities
        id_e1 = p.attributes["e1"].value
        id_e2 = p.attributes["e2"].value

In [8]:
def traverse_path(path, tree):
    if len(path) == 0:
        return None
    path_nodes = [tree.nodes[x] for x in path]
    str_path = ""
    # traverse from e1 up
    current_node = path_nodes[0]
    while (current_node['head'] in path):
        
        rel = current_node['rel']
        current_node = tree.nodes[current_node['head']]
        str_path += (rel + '<')
    
    str_path += current_node['lemma']
    # traverse from e2 up
    current_node = path_nodes[-1]
    while(current_node['head'] in path):
        rel = current_node['rel']
        current_node = tree.nodes[current_node['head']]
        str_path += ('>' + rel)
        
    return str_path
    
    

In [9]:
CLUE_VERBS = ['administer', 'enhance', 'interact', 'coadminister', 'increase', 'decrease'] # add more?

def find_clue_verbs(path, tree):
    path_nodes = [tree.nodes[x]['lemma'] for x in path]
    feats = []
    for pn in path_nodes:
        if pn in CLUE_VERBS:
            feats.append('lemmainbetween=%s' % pn)
            
    return feats

In [14]:
def extract_features(tree, entities, e1, e2) :
    '''
    Task:
        Given an analyzed sentence and two target entities , compute a feature
        vector for this classification example .
    Input:
        tree: a DependencyGraph object with all sentence information .
        entities: A list of all entities in the sentence (id and offsets).
        e1, e2: ids of the two entities to be checked for an interaction
    Output:
        A vector of binary features .
        Features are binary and vectors are in sparse representation (i.e. only
        active features are listed)
   '''
    
    
        
    e1_node = find_entity_in_tree(e1, entities, tree)
    e2_node = find_entity_in_tree(e2, entities, tree)
    
    e1_head = find_head(tree, e1_node) if e1_node else None
    e2_head = find_head(tree, e2_node) if e2_node else None
    
    h1_lemma = e1_head['lemma'] if e1_node else None
    h2_lemma = e2_head['lemma'] if e2_node else None
    

    
    tag_head_e1 = e1_head['tag'] if e1_head else None
    tag_head_e2 = e2_head['tag'] if e2_head else None
    
    nxgraph = tree.nx_graph().to_undirected()
    shortest_path = networkx.shortest_path(nxgraph, e1_node['address'], e2_node['address']) if (e1_node and e2_node) else []
    path = traverse_path(shortest_path, analysis)
    find_clue_verbs(shortest_path, analysis)

    
    #e1_tag = e1_node['tag'] if e1_node else None
    #e2_tag = e2_node['tag'] if e2_node else None
    
    #h_e1 = e1_node['head'] if e1_node else None      
    #l_h1 = analysis.nodes[h_e1]['lemma'] if e1_node else None
    #w_h1 = analysis.nodes[h_e1]['word'] if e1_node else None
    #tag_head_e1 = analysis.nodes[h_e1]['tag'][0].lower() if e1_node else None
    
    #h_e2 = e2_node['head'] if e2_node else None
    #l_h2 = analysis.nodes[h_e2]['lemma'] if e2_node else None
    #tag_head_e2 = analysis.nodes[h_e2]['tag'][0].lower() if e2_node else None
    
    # --- FEATURES ---
    features = ['h1_lemma=%s' %h1_lemma,
                'h2_lemma=%s' %h2_lemma,
                'h1_tag=%s' %tag_head_e1,
                'h2_tag=%s' %tag_head_e2,
                'path=%s' % path
                
                ] + find_clue_verbs(shortest_path, analysis)
    return features


In [15]:
# file for initial checks
file = '/Users/mponsclo/Documents/Master/labAHLT/data/train/3155550.xml'
file = '../../labAHLT/data/train/Dexamethasone_ddi.xml'
tree = parse(file)
sentences = tree.getElementsByTagName("sentence")
for s in sentences:
    sid = s.attributes["id"].value
    stext = s.attributes["text"].value
    
    entities = {}
    ents = s.getElementsByTagName("entity")
    for e in ents:
        eid = e . attributes["id"].value
        entities[eid] = e.attributes["charOffset"].value.split("-")
    if len(entities) > 1: analysis = analyze(stext)
    
    pairs = s.getElementsByTagName("pair")
    for p in pairs:
        # get ground truth
        ddi = p.attributes["ddi"].value
        dditype = p.attributes["type"].value if ddi == "true" else "null"
            
        # target entities
        id_e1 = p.attributes["e1"].value
        id_e2 = p.attributes["e2"].value
        extract_features(analysis, entities, id_e1, id_e2)

In [17]:
datadir = "../../labAHLT/data/devel"

outfile = "devel_feats.dat"

# process each file in directory
for f in listdir(datadir):
    # parse XML file , obtaining a DOM tree
    tree = parse(datadir + "/" + f)
    # process each sentence in the file
    sentences = tree.getElementsByTagName("sentence")
    for s in sentences:

        sid = s.attributes["id"].value # get sentence id
        stext = s.attributes["text"].value # get sentence text
        
        # CoreNLP throws error for empty sentences
        if len(stext) == 0:
            continue

        # load sentence ground truth entities
        entities = {}
        ents = s.getElementsByTagName("entity")
        for e in ents:
            eid = e . attributes["id"].value
            entities[eid] = e.attributes["charOffset"].value.split("-")

        # analyze sentence if there is at least a pair of entities
        if len(entities) > 1: analysis = analyze(stext)

        # for each pair in the sentence , decide whether it is DDI and its type
        pairs = s.getElementsByTagName("pair")
        for p in pairs:
            # get ground truth
            ddi = p.attributes["ddi"].value
            dditype = p.attributes["type"].value if ddi == "true" else "null"
            
            # target entities
            id_e1 = p.attributes["e1"].value
            id_e2 = p.attributes["e2"].value
            
            # feature extraction
            feats = extract_features(analysis, entities, id_e1, id_e2)
            
            # resulting feature vector
            print(sid, id_e1, id_e2, dditype, "|".join(feats), sep="|") 
            

DDI-DrugBank.d105.s0|DDI-DrugBank.d105.s0.e0|DDI-DrugBank.d105.s0.e1|null|h1_lemma=increase|h2_lemma=aggregation|h1_tag=VB|h2_tag=NN|path=anticoagulant>compound>compound>conj>nmod
DDI-DrugBank.d105.s0|DDI-DrugBank.d105.s0.e0|DDI-DrugBank.d105.s0.e2|null|h1_lemma=increase|h2_lemma=drug|h1_tag=VB|h2_tag=NNS|path=anticoagulant>amod>nmod>conj>nmod
DDI-DrugBank.d105.s0|DDI-DrugBank.d105.s0.e0|DDI-DrugBank.d105.s0.e3|null|h1_lemma=increase|h2_lemma=drug|h1_tag=VB|h2_tag=NNS|path=anticoagulant>dep>nmod>conj>nmod
DDI-DrugBank.d105.s0|DDI-DrugBank.d105.s0.e0|DDI-DrugBank.d105.s0.e4|null|h1_lemma=increase|h2_lemma=drug|h1_tag=VB|h2_tag=NNS|path=anticoagulant>conj>nmod>conj>nmod
DDI-DrugBank.d105.s0|DDI-DrugBank.d105.s0.e0|DDI-DrugBank.d105.s0.e5|effect|h1_lemma=increase|h2_lemma=administer|h1_tag=VB|h2_tag=VBN|path=nsubj<increase>obl>dep>advmod|lemmainbetween=increase|lemmainbetween=administer
DDI-DrugBank.d105.s0|DDI-DrugBank.d105.s0.e1|DDI-DrugBank.d105.s0.e2|null|h1_lemma=aggregation|h2_lemma

DDI-MedLine.d23.s2|DDI-MedLine.d23.s2.e2|DDI-MedLine.d23.s2.e4|null|h1_lemma=Innovar|h2_lemma=administration|h1_tag=NNP|h2_tag=NN|path=conj<nmod<obl<convert>nmod>obl>dep
DDI-MedLine.d23.s2|DDI-MedLine.d23.s2.e2|DDI-MedLine.d23.s2.e5|null|h1_lemma=Innovar|h2_lemma=administration|h1_tag=NNP|h2_tag=NN|path=conj<nmod<obl<convert>conj>obl>dep
DDI-MedLine.d23.s2|DDI-MedLine.d23.s2.e3|DDI-MedLine.d23.s2.e4|null|h1_lemma=Innovar|h2_lemma=administration|h1_tag=NNP|h2_tag=NN|path=conj<nmod<obl<convert>nmod>obl>dep
DDI-MedLine.d23.s2|DDI-MedLine.d23.s2.e3|DDI-MedLine.d23.s2.e5|null|h1_lemma=Innovar|h2_lemma=administration|h1_tag=NNP|h2_tag=NN|path=conj<nmod<obl<convert>conj>obl>dep
DDI-MedLine.d23.s2|DDI-MedLine.d23.s2.e4|DDI-MedLine.d23.s2.e5|null|h1_lemma=administration|h2_lemma=administration|h1_tag=NN|h2_tag=NN|path=nmod<administration>conj
DDI-DrugBank.d287.s0|DDI-DrugBank.d287.s0.e0|DDI-DrugBank.d287.s0.e1|advise|h1_lemma=administer|h2_lemma=antagonist|h1_tag=VBN|h2_tag=NNS|path=nsubj:pass<

DDI-DrugBank.d179.s12|DDI-DrugBank.d179.s12.e0|DDI-DrugBank.d179.s12.e1|null|h1_lemma=None|h2_lemma=divalproex|h1_tag=TOP|h2_tag=NN|path=divalproex>dep
DDI-DrugBank.d179.s14|DDI-DrugBank.d179.s14.e0|DDI-DrugBank.d179.s14.e1|null|h1_lemma=None|h2_lemma=etretinate|h1_tag=TOP|h2_tag=NN|path=etretinate>appos
DDI-DrugBank.d179.s16|DDI-DrugBank.d179.s16.e0|DDI-DrugBank.d179.s16.e1|null|h1_lemma=None|h2_lemma=hydroxychloroquine|h1_tag=TOP|h2_tag=NN|path=hydroxychloroquine>appos
DDI-DrugBank.d179.s17|DDI-DrugBank.d179.s17.e0|DDI-DrugBank.d179.s17.e1|null|h1_lemma=None|h2_lemma=mercaptopurine|h1_tag=TOP|h2_tag=NN|path=mercaptopurine>dep
DDI-DrugBank.d179.s18|DDI-DrugBank.d179.s18.e0|DDI-DrugBank.d179.s18.e1|null|h1_lemma=None|h2_lemma=naltrexone|h1_tag=TOP|h2_tag=NN|path=naltrexone>appos
DDI-DrugBank.d179.s19|DDI-DrugBank.d179.s19.e0|DDI-DrugBank.d179.s19.e1|null|h1_lemma=None|h2_lemma=contain|h1_tag=TOP|h2_tag=VBG|path=contraceptive>obj>acl
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e0|DDI-Dr

DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e3|DDI-DrugBank.d179.s21.e8|null|h1_lemma=phenothiazine|h2_lemma=mesoridazine|h1_tag=NNS|h2_tag=NN|path=conj<phenothiazine>appos>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e3|DDI-DrugBank.d179.s21.e9|null|h1_lemma=phenothiazine|h2_lemma=phenothiazine|h1_tag=NNS|h2_tag=NNS|path=conj<phenothiazine>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e3|DDI-DrugBank.d179.s21.e10|null|h1_lemma=phenothiazine|h2_lemma=perphenazine|h1_tag=NNS|h2_tag=NN|path=conj<phenothiazine>appos>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e3|DDI-DrugBank.d179.s21.e11|null|h1_lemma=phenothiazine|h2_lemma=phenothiazine|h1_tag=NNS|h2_tag=NNS|path=conj<phenothiazine>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e3|DDI-DrugBank.d179.s21.e12|null|h1_lemma=phenothiazine|h2_lemma=prochlorperazine|h1_tag=NNS|h2_tag=NN|path=conj<phenothiazine>appos>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e3|DDI-DrugBank.d179.s21.e13|null|h1_lemma=phenothiazine|h2_lemma=N

DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e16|DDI-DrugBank.d179.s21.e23|null|h1_lemma=promethazine|h2_lemma=phenothiazine|h1_tag=NN|h2_tag=NNS|path=appos<conj<phenothiazine>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e16|DDI-DrugBank.d179.s21.e24|null|h1_lemma=promethazine|h2_lemma=trimeprazine|h1_tag=NN|h2_tag=NN|path=appos<conj<phenothiazine>appos>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e17|DDI-DrugBank.d179.s21.e18|null|h1_lemma=phenothiazine|h2_lemma=thioridazine|h1_tag=NNS|h2_tag=NN|path=thioridazine>appos
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e17|DDI-DrugBank.d179.s21.e19|null|h1_lemma=phenothiazine|h2_lemma=phenothiazine|h1_tag=NNS|h2_tag=NNS|path=conj<phenothiazine>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e17|DDI-DrugBank.d179.s21.e20|null|h1_lemma=phenothiazine|h2_lemma=trifluoperazine|h1_tag=NNS|h2_tag=NN|path=conj<phenothiazine>appos>conj
DDI-DrugBank.d179.s21|DDI-DrugBank.d179.s21.e17|DDI-DrugBank.d179.s21.e21|null|h1_lemma=phenothiazine|h2_lem

...

DDI-DrugBank.d197.s2|DDI-DrugBank.d197.s2.e2|DDI-DrugBank.d197.s2.e3|null|h1_lemma=sevoflurane|h2_lemma=sevoflurane|h1_tag=NN|h2_tag=NN|path=conj<sevoflurane>conj
DDI-DrugBank.d197.s2|DDI-DrugBank.d197.s2.e2|DDI-DrugBank.d197.s2.e4|null|h1_lemma=sevoflurane|h2_lemma=sevoflurane|h1_tag=NN|h2_tag=NN|path=conj<sevoflurane>conj
DDI-DrugBank.d197.s2|DDI-DrugBank.d197.s2.e3|DDI-DrugBank.d197.s2.e4|null|h1_lemma=sevoflurane|h2_lemma=sevoflurane|h1_tag=NN|h2_tag=NN|path=conj<sevoflurane>conj
DDI-DrugBank.d197.s3|DDI-DrugBank.d197.s3.e0|DDI-DrugBank.d197.s3.e1|null|h1_lemma=interaction|h2_lemma=dexmedetomidine|h1_tag=NNS|h2_tag=NN|path=dexmedetomidine>conj
DDI-DrugBank.d197.s3|DDI-DrugBank.d197.s3.e0|DDI-DrugBank.d197.s3.e2|null|h1_lemma=interaction|h2_lemma=dexmedetomidine|h1_tag=NNS|h2_tag=NN|path=dexmedetomidine>conj
DDI-DrugBank.d197.s3|DDI-DrugBank.d197.s3.e0|DDI-DrugBank.d197.s3.e3|null|h1_lemma=interaction|h2_lemma=dexmedetomidine|h1_tag=NNS|h2_tag=NN|path=dexmedetomidine>conj
DDI-DrugBa

DDI-DrugBank.d465.s0|DDI-DrugBank.d465.s0.e0|DDI-DrugBank.d465.s0.e1|effect|h1_lemma=dinitrate|h2_lemma=those|h1_tag=NN|h2_tag=DT|path=compound<nmod<nsubj<additive>nmod>obl
DDI-DrugBank.d95.s0|DDI-DrugBank.d95.s0.e0|DDI-DrugBank.d95.s0.e1|null|h1_lemma=arginine|h2_lemma=arginine|h1_tag=NN|h2_tag=NN|path=compound<arginine>compound
DDI-DrugBank.d95.s0|DDI-DrugBank.d95.s0.e0|DDI-DrugBank.d95.s0.e2|null|h1_lemma=arginine|h2_lemma=effect|h1_tag=NN|h2_tag=NN|path=compound<nsubj<counteract>nmod>obj
DDI-DrugBank.d95.s0|DDI-DrugBank.d95.s0.e1|DDI-DrugBank.d95.s0.e2|effect|h1_lemma=arginine|h2_lemma=effect|h1_tag=NN|h2_tag=NN|path=compound<nsubj<counteract>nmod>obj
DDI-DrugBank.d95.s1|DDI-DrugBank.d95.s1.e0|DDI-DrugBank.d95.s1.e1|null|h1_lemma=arginine|h2_lemma=arginine|h1_tag=NN|h2_tag=NN|path=compound<arginine>compound
DDI-DrugBank.d95.s1|DDI-DrugBank.d95.s1.e0|DDI-DrugBank.d95.s1.e2|null|h1_lemma=arginine|h2_lemma=absorption|h1_tag=NN|h2_tag=NN|path=compound<nsubj<increase>nmod>obj|lemmainbet

DDI-DrugBank.d237.s8|DDI-DrugBank.d237.s8.e0|DDI-DrugBank.d237.s8.e1|mechanism|h1_lemma=inhibit|h2_lemma=metabolism|h1_tag=VBZ|h2_tag=NN|path=nsubj<inhibit>nmod>obj
DDI-DrugBank.d237.s8|DDI-DrugBank.d237.s8.e0|DDI-DrugBank.d237.s8.e2|null|h1_lemma=inhibit|h2_lemma=None|h1_tag=VBZ|h2_tag=None|path=None
DDI-DrugBank.d237.s8|DDI-DrugBank.d237.s8.e1|DDI-DrugBank.d237.s8.e2|null|h1_lemma=metabolism|h2_lemma=None|h1_tag=NN|h2_tag=None|path=None
DDI-DrugBank.d237.s9|DDI-DrugBank.d237.s9.e0|DDI-DrugBank.d237.s9.e1|effect|h1_lemma=coadministration|h2_lemma=cisapride|h1_tag=NN|h2_tag=NN|path=cisapride>conj
DDI-DrugBank.d237.s10|DDI-DrugBank.d237.s10.e0|DDI-DrugBank.d237.s10.e1|null|h1_lemma=antagonist|h2_lemma=coadministration|h1_tag=NNS|h2_tag=NN|path=compound<antagonist>compound>nsubj>appos
DDI-DrugBank.d237.s10|DDI-DrugBank.d237.s10.e0|DDI-DrugBank.d237.s10.e2|null|h1_lemma=antagonist|h2_lemma=concentration|h1_tag=NNS|h2_tag=NN|path=compound<antagonist>nmod>obl>appos
DDI-DrugBank.d237.s10|DDI

DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e0|DDI-DrugBank.d141.s0.e1|null|h1_lemma=drug|h2_lemma=derivative|h1_tag=NNS|h2_tag=NNS|path=amod<drug>compound>conj
DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e0|DDI-DrugBank.d141.s0.e2|null|h1_lemma=drug|h2_lemma=drug|h1_tag=NNS|h2_tag=NNS|path=amod<drug>appos
DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e0|DDI-DrugBank.d141.s0.e3|null|h1_lemma=drug|h2_lemma=Acid|h1_tag=NNS|h2_tag=NNP|path=amod<drug>compound>dep>appos
DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e0|DDI-DrugBank.d141.s0.e4|null|h1_lemma=drug|h2_lemma=Acid|h1_tag=NNS|h2_tag=NNP|path=amod<drug>appos>dep>appos
DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e0|DDI-DrugBank.d141.s0.e5|null|h1_lemma=drug|h2_lemma=Antifungals|h1_tag=NNS|h2_tag=NNPS|path=amod<drug>compound>appos>dep>appos
DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e1|DDI-DrugBank.d141.s0.e2|null|h1_lemma=derivative|h2_lemma=drug|h1_tag=NNS|h2_tag=NNS|path=compound<conj<drug>appos
DDI-DrugBank.d141.s0|DDI-DrugBank.d141.s0.e1|DDI-Dr

DDI-DrugBank.d141.s12|DDI-DrugBank.d141.s12.e0|DDI-DrugBank.d141.s12.e1|null|h1_lemma=None|h2_lemma=auc|h1_tag=TOP|h2_tag=NN|path=ERYTHROMYCIN>compound>appos
DDI-DrugBank.d141.s12|DDI-DrugBank.d141.s12.e0|DDI-DrugBank.d141.s12.e2|null|h1_lemma=None|h2_lemma=None|h1_tag=TOP|h2_tag=None|path=None
DDI-DrugBank.d141.s12|DDI-DrugBank.d141.s12.e1|DDI-DrugBank.d141.s12.e2|mechanism|h1_lemma=auc|h2_lemma=None|h1_tag=NN|h2_tag=None|path=None
DDI-DrugBank.d141.s13|DDI-DrugBank.d141.s13.e0|DDI-DrugBank.d141.s13.e1|null|h1_lemma=sodium|h2_lemma=convert|h1_tag=NN|h2_tag=VBG|path=compound<nsubj:pass<use>obl>amod>obl
DDI-DrugBank.d141.s13|DDI-DrugBank.d141.s13.e0|DDI-DrugBank.d141.s13.e2|null|h1_lemma=sodium|h2_lemma=inhibitor|h1_tag=NN|h2_tag=NNS|path=compound<nsubj:pass<use>conj>dep>obl
DDI-DrugBank.d141.s13|DDI-DrugBank.d141.s13.e0|DDI-DrugBank.d141.s13.e3|null|h1_lemma=sodium|h2_lemma=channel|h1_tag=NN|h2_tag=NN|path=compound<nsubj:pass<use>compound>compound>conj>dep>obl
DDI-DrugBank.d141.s13|DDI